In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# data directory with images
train_image_dir = 'sampled_data/train_data'
val_image_dir = 'sampled_data/val_data'
test_image_dir = 'sampled_data/test_data'

# Load metadata from csv files (only 'filename' and 'age' columns)
train_metadata = pd.read_csv('sampled_data/train_data.csv', usecols=['filename', 'age'])
val_metadata = pd.read_csv('sampled_data/val_data.csv', usecols=['filename', 'age'])
test_metadata = pd.read_csv('sampled_data/test_data.csv', usecols=['filename', 'age'])


In [3]:
# Define image size - in resnet 224 by 224 is the norm?
image_size = (224, 224)

# load and preprocess images like resize, normalising pixel values 
def load_image(filename, image_dir):
    image = cv2.imread(os.path.join(image_dir, filename))
    image = cv2.resize(image, image_size)
    image = image.astype('float32') / 255.0  # Normalize pixel values
    return image

In [4]:
# Load images for training, validation, and testing
train_images = np.array([load_image(filename, train_image_dir) for filename in train_metadata['filename']])
val_images = np.array([load_image(filename, val_image_dir) for filename in val_metadata['filename']])
test_images = np.array([load_image(filename, test_image_dir) for filename in test_metadata['filename']])

KeyboardInterrupt: 

In [39]:
# Extract labels
train_labels = train_metadata['age'].values
val_labels = val_metadata['age'].values
test_labels = test_metadata['age'].values

In [40]:
# Load pre-trained ResNet50 model without top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

94765736/94765736 [==============================] - 6s 0us/step


In [41]:
# Freeze the pre-trained layers - prevent them from being updated during training. Set layer.trainable = False for each layer in the base model ! 
for layer in base_model.layers:
    layer.trainable = False

In [42]:
# Add custom top layers for age prediction, use relu activation
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1)(x)

In [43]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [44]:
# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

In [47]:
# Train the model
model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10, batch_size=32)

MemoryError: Unable to allocate 16.8 GiB for an array with shape (29913, 224, 224, 3) and data type float32